In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

from tensor2tensor.data_generators import problem
from tensor2tensor.data_generators import text_problems
from tensor2tensor.data_generators import translate
from tensor2tensor.utils import registry
from tensor2tensor import problems
import tensorflow as tf
import os
import logging

logger = logging.getLogger()
tf.logging.set_verbosity(tf.logging.DEBUG)

In [2]:
import youtokentome as yttm

bpe = yttm.BPE(model='segmentation.yttm')

class Encoder:
    def __init__(self, bpe):
        self.bpe = bpe
        self.vocab_size = len(self.bpe.vocab())
    
    def encode(self, s):
        s = self.bpe.encode(s, output_type=yttm.OutputType.ID)
        s = [i + [1] for i in s]
        return s
    
    def decode(self, ids, strip_extraneous=False):
        ids = [[k for k in i if k > 1] for i in ids]
        return self.bpe.decode(list(ids))
    
encoder = Encoder(bpe)

In [3]:
@registry.register_problem
class Segmentation(text_problems.Text2TextProblem):
    @property
    def approx_vocab_size(self):
        return 32000

    @property
    def is_generate_per_split(self):
        return False

    def feature_encoders(self, data_dir):
        encoder = Encoder(bpe)
        return {'inputs': encoder, 'targets': encoder}

In [4]:
DATA_DIR = os.path.expanduser('t2t-segmentation/data')
TMP_DIR = os.path.expanduser('t2t-segmentation/tmp')
TRAIN_DIR = os.path.expanduser('t2t-segmentation/train-base')

In [5]:
PROBLEM = 'segmentation'
t2t_problem = problems.problem(PROBLEM)

train_steps = 500000
eval_steps = 10
batch_size = 4096
save_checkpoints_steps = 25000
ALPHA = 0.1
schedule = 'continuous_train_and_eval'
MODEL = 'transformer'
HPARAMS = 'transformer_base'

In [6]:
import tensorflow as tf
import os

ckpt_path = tf.train.latest_checkpoint(TRAIN_DIR)
ckpt_path

't2t-segmentation/train-base/model.ckpt-200000'

In [7]:
from tensor2tensor import models
from tensor2tensor import problems
from tensor2tensor.layers import common_layers
from tensor2tensor.utils import trainer_lib
from tensor2tensor.utils import t2t_model
from tensor2tensor.utils import registry
from tensor2tensor.utils import metrics
from tensor2tensor.data_generators import problem
from tensor2tensor.data_generators import text_problems
from tensor2tensor.data_generators import translate
from tensor2tensor.utils import registry

In [8]:
class Model:
    def __init__(self, HPARAMS = "transformer_base", DATA_DIR = 't2t/data'):
        
        self.X = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None, None])
        
        self.X_seq_len = tf.count_nonzero(self.X, 1, dtype=tf.int32)
        maxlen_decode = tf.reduce_max(self.X_seq_len)
        maxlen_decode = 50 + tf.reduce_max(self.X_seq_len)
        #self.maxlen_decode = tf.placeholder(tf.int32, None)
        
        x = tf.expand_dims(tf.expand_dims(self.X, -1), -1)
        y = tf.expand_dims(tf.expand_dims(self.Y, -1), -1)
        
        features = {
            "inputs": x,
            "targets": y,
            "target_space_id": tf.constant(1, dtype=tf.int32),
        }
        self.features = features
        
        Modes = tf.estimator.ModeKeys
        hparams = trainer_lib.create_hparams(HPARAMS, data_dir=DATA_DIR, problem_name=PROBLEM)
        hparams.max_length = 512
        
        translate_model = registry.model('transformer')(hparams, Modes.PREDICT)
        self.translate_model = translate_model
        logits, _ = translate_model(features)
        self.logits = logits
        
        with tf.variable_scope(tf.get_variable_scope(), reuse=True):
            self.fast_result = translate_model._greedy_infer(features, maxlen_decode)["outputs"]
            self.beam_result = translate_model._beam_decode_slow(
                features, maxlen_decode, beam_size=3, 
                top_beams=1, alpha=0.5)["outputs"]
        
        self.fast_result = tf.identity(self.fast_result, name = 'greedy')
        self.beam_result = tf.identity(self.beam_result, name = 'beam')
        
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model()
var_lists = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
saver = tf.train.Saver(var_list = var_lists)
saver.restore(sess, ckpt_path)

Instructions for updating:
reduction_indices is deprecated, use axis instead


Instructions for updating:
reduction_indices is deprecated, use axis instead


INFO:tensorflow:Setting T2TModel mode to 'infer'


INFO:tensorflow:Setting T2TModel mode to 'infer'


INFO:tensorflow:Setting hparams.dropout to 0.0


...

INFO:tensorflow:Building model body


INFO:tensorflow:Transforming body output with symbol_modality_32000_512.top


INFO:tensorflow:Transforming body output with symbol_modality_32000_512.top


INFO:tensorflow:Restoring parameters from t2t-segmentation/train-base/model.ckpt-200000


INFO:tensorflow:Restoring parameters from t2t-segmentation/train-base/model.ckpt-200000


In [9]:
def padding_sequence(seq, maxlen = None, padding = 'post', pad_int = 0):
    if not maxlen:
        maxlen = max([len(i) for i in seq])
    padded_seqs = []
    for s in seq:
        if padding == 'post':
            padded_seqs.append(s + [pad_int] * (maxlen - len(s)))
        if padding == 'pre':
            padded_seqs.append([pad_int] * (maxlen - len(s)) + s)
    return padded_seqs

In [10]:
string1 = 'huseinsukamakan ayam,dia sgtrisaukan'
string2 = 'drmahathir sangat menekankan budaya budakzamansekarang'
string3 = 'ceritatunnajibrazak'
string4 = 'TunM sukakan'

In [11]:
encoded = padding_sequence(encoder.encode([string1, string2, string3, string4]))
greedy = sess.run(model.fast_result, 
             feed_dict = {model.X: encoded})
encoder.decode(greedy)

['husein suka makan ayam, dia sgt risaukan',
 'dr mahathir sangat menekankan budaya budak zaman sekarang',
 'cerita tun najib razak',
 'Tun M sukakan Tun M sukakan']

In [12]:
saver = tf.train.Saver(tf.trainable_variables())
saver.save(sess, 'segmentation-base/model.ckpt')

'segmentation-base/model.ckpt'

In [13]:
strings = ','.join(
    [
        n.name
        for n in tf.get_default_graph().as_graph_def().node
        if ('Variable' in n.op
        or 'Placeholder' in n.name
        or 'greedy' in n.name
        or 'beam' in n.name
        or 'alphas' in n.name
        or 'self/Softmax' in n.name)
        and 'adam' not in n.name
        and 'beta' not in n.name
        and 'global_step' not in n.name
        and 'modality' not in n.name
        and 'Assign' not in n.name
    ]
)

In [14]:
def freeze_graph(model_dir, output_node_names):

    if not tf.gfile.Exists(model_dir):
        raise AssertionError(
            "Export directory doesn't exists. Please specify an export "
            'directory: %s' % model_dir
        )

    checkpoint = tf.train.get_checkpoint_state(model_dir)
    input_checkpoint = checkpoint.model_checkpoint_path

    absolute_model_dir = '/'.join(input_checkpoint.split('/')[:-1])
    output_graph = absolute_model_dir + '/frozen_model.pb'
    clear_devices = True
    with tf.Session(graph = tf.Graph()) as sess:
        saver = tf.train.import_meta_graph(
            input_checkpoint + '.meta', clear_devices = clear_devices
        )
        saver.restore(sess, input_checkpoint)
        output_graph_def = tf.graph_util.convert_variables_to_constants(
            sess,
            tf.get_default_graph().as_graph_def(),
            output_node_names.split(','),
        )
        with tf.gfile.GFile(output_graph, 'wb') as f:
            f.write(output_graph_def.SerializeToString())
        print('%d ops in the final graph.' % len(output_graph_def.node))

In [15]:
freeze_graph('segmentation-base', strings)

INFO:tensorflow:Restoring parameters from segmentation-base/model.ckpt


INFO:tensorflow:Restoring parameters from segmentation-base/model.ckpt


Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`


Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`


Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


...

INFO:tensorflow:Froze 201 variables.


INFO:tensorflow:Froze 201 variables.


INFO:tensorflow:Converted 201 variables to const ops.


INFO:tensorflow:Converted 201 variables to const ops.


11006 ops in the final graph.


In [16]:
def load_graph(frozen_graph_filename):
    with tf.gfile.GFile(frozen_graph_filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def)
    return graph

In [17]:
g = load_graph('segmentation-base/frozen_model.pb')
x = g.get_tensor_by_name('import/Placeholder:0')
greedy = g.get_tensor_by_name('import/greedy:0')
beam = g.get_tensor_by_name('import/beam:0')
test_sess = tf.InteractiveSession(graph = g)

/home/husein/.local/lib/python3.6/site-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [18]:
g, b = test_sess.run([greedy, beam], feed_dict = {x:encoded})

In [19]:
encoder.decode(g)

['husein suka makan ayam, dia sgt risaukan',
 'dr mahathir sangat menekankan budaya budak zaman sekarang',
 'cerita tun najib razak',
 'Tun M sukakan Tun M sukakan']

In [20]:
encoder.decode(b)

['husein suka makan ayam, dia sgt risaukan',
 'dr mahathir sangat menekankan budaya budak zaman sekarang',
 'cerita tun najib razak',
 'Tun M sukakan']

In [21]:
from tensorflow.tools.graph_transforms import TransformGraph
from glob import glob
tf.set_random_seed(0)

In [22]:
import tensorflow_text
import tf_sentencepiece

In [23]:
transforms = ['add_default_attributes',
             'remove_nodes(op=Identity, op=CheckNumerics, op=Dropout)',
             'fold_batch_norms',
             'fold_old_batch_norms',
             'quantize_weights(fallback_min=-10, fallback_max=10)',
             'strip_unused_nodes',
             'sort_by_execution_order']

pb = 'segmentation-base/frozen_model.pb'
input_graph_def = tf.GraphDef()
with tf.gfile.FastGFile(pb, 'rb') as f:
    input_graph_def.ParseFromString(f.read())

transformed_graph_def = TransformGraph(input_graph_def, 
                                       ['Placeholder'],
                                       ['greedy', 'beam'], transforms)

with tf.gfile.GFile(f'{pb}.quantized', 'wb') as f:
    f.write(transformed_graph_def.SerializeToString())

Instructions for updating:
Use tf.gfile.GFile.


Instructions for updating:
Use tf.gfile.GFile.
